In [1]:
#import necessary packages
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import re

In [2]:
#import necessary packages for further word processing
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2020)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /global/homes/d/dsmorrow/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
#function to perform lemmatize and stem preprocessing steps on the data set.
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 1:
            result.append(lemmatize_stemming(token))
    return result

In [12]:
full_proc = pd.read_csv("Full_Table_ICD9_Notes.csv", usecols=["text_processed", "index"])
full_proc

,text_processed,index
0,admission date 2142 5 15 discharge date 2142 5...,0
1,admission date 2142 5 20 discharge date 2142 6...,1
2,admission date 2142 6 18 discharge date 2142 6...,2
3,admission date 2142 7 3 discharge date 2142 7 ...,3
4,admission date 2142 7 7 discharge date 2142 7 ...,4
...,...,...
323055,last name lf first name3 lf 1046 j last name ...,323055
323056,2143 9 3 9 59 am chest pa lat clip clip numbe...,323056
323057,2144 2 25 1 49 pm ankle ap mortise lat left c...,323057
323058,2144 1 7 4 21 pm ankle ap mortise lat left cl...,323058


In [4]:
#remove numbers
full_proc['text_processed'] = full_proc['text_processed'].map(lambda x: re.sub('(\s\d+)', ' ', x))

In [5]:
documents = full_proc

In [12]:
#use preprocess function
processed_docs = documents['text_processed'].map(preprocess)
processed_docs

In [ ]:
#processed_docs.to_csv('Full_Text_Processed.csv')

In [4]:
#load in already processed csv, so we dont have to wait for first preprocessing
full_proc2 = pd.read_csv("Full_Text_Processed.csv")
full_proc2

,Unnamed: 0,text_processed
0,0,"['admiss', 'date', 'discharg', 'date', 'date',..."
1,1,"['admiss', 'date', 'discharg', 'date', 'date',..."
2,2,"['admiss', 'date', 'discharg', 'date', 'date',..."
3,3,"['admiss', 'date', 'discharg', 'date', 'date',..."
4,4,"['admiss', 'date', 'discharg', 'date', 'date',..."
...,...,...
432684,432684,"['titl', 'woman', 'histori', 'headach', 'talk'..."
432685,432685,"['sicu', 'woman', 'depress', 'wellbutrin', 'he..."
432686,432686,"['head', 'contrast', 'clip', 'clip', 'number',..."
432687,432687,"['head', 'recon', 'neck', 'recon', 'clip', 'cl..."


In [5]:
def preprocess2(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 1:
            result.append(token)
    return result

In [6]:
#make an interable again so can be make into dictionary
full_proc2 = full_proc2['text_processed'].map(preprocess2)

In [7]:
documents = full_proc2

In [8]:
#create dictionary of words and number of appearances 
dictionary = gensim.corpora.Dictionary(documents)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abdomen
1 abdomin
2 abl
3 abnorm
4 abus
5 accid
6 acut
7 admiss
8 admit
9 afebril
10 agre


In [9]:
#filter out words that appear in less than (15) documents
#only keep the first 10000
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [10]:
#go through each document and report words and occurrences using doc2box for token id and amount
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

In [11]:
#bow_corpus

In [194]:
#determine the TF-IDF scores or weight of a word within the documents
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(sorted(doc, key = lambda x: x[1]))
    break

[(71, 0.015626410521867647),
 (78, 0.015844633004390183),
 (64, 0.019595060068982775),
 (17, 0.021715677003400876),
 (15, 0.02213536108092899),
 (84, 0.022768788765754895),
 (39, 0.023040852787759327),
 (41, 0.023576100652745527),
 (75, 0.02370372807995732),
 (25, 0.02411900265952652),
 (67, 0.024564054242296768),
 (81, 0.02961680137263958),
 (31, 0.030558786652149394),
 (73, 0.03235680953754377),
 (53, 0.0325123726474259),
 (85, 0.0332739724055729),
 (49, 0.03333030469600802),
 (59, 0.034013788185536145),
 (0, 0.035226667563784696),
 (40, 0.036302306550574155),
 (48, 0.03708293462008571),
 (94, 0.037646636031132705),
 (83, 0.0379492642243804),
 (16, 0.03801746581083953),
 (30, 0.04242689414658946),
 (61, 0.04297792221285993),
 (95, 0.044670114221930596),
 (46, 0.04476522492698133),
 (65, 0.04502607008431103),
 (42, 0.045189102156805674),
 (21, 0.04540441530260847),
 (88, 0.045463536605562516),
 (90, 0.04664184677327331),
 (19, 0.04782633832705319),
 (7, 0.04804934754015209),
 (79, 0.0

In [14]:
#top ten weighted words from full documents dictionary and BOW
print(dictionary[306] ,"," + dictionary[345] ,"," + dictionary[223] ,"," + dictionary[321] ,"," + dictionary[302] ,"," + dictionary[251] ,"," + dictionary[333],"," + dictionary[260],"," + dictionary[209],"," + dictionary[87])

tablet ,weekli ,patch ,transderm ,sustain ,qwed ,urgenc ,releas ,nifedipin ,dialysi


In [28]:
#top ten weighted words from 50k documents
print(dictionary[307] ,"," + dictionary[345] ,"," + dictionary[224] ,"," + dictionary[303] ,"," + dictionary[321] ,"," + dictionary[261] ,"," + dictionary[76],"," + dictionary[252],"," + dictionary[333],"," + dictionary[93])

tablet ,weekli ,patch ,sustain ,transderm ,releas ,daili ,qwed ,urgenc ,discharg


In [13]:
# Gensim extras so we can plot word weights
import gensim.corpora as corpora
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [66]:
#train LDA model using BOW, chunk size is 5000 documents, lda is updated after every chunk size, 2 full passes through the corpus for training, produce 10 topics
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=5000,
                                           passes=2)

In [67]:
# Print the Keywords in the 10 topics using the BOW corpus
pprint(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]

[(0,
  '0.034*"normal" + 0.029*"valv" + 0.024*"leav" + 0.020*"aortic" + '
  '0.019*"ventricular" + 0.014*"right" + 0.014*"mitral" + 0.014*"leaflet" + '
  '0.012*"size" + 0.011*"wall"'),
 (1,
  '0.014*"patient" + 0.012*"tablet" + 0.009*"daili" + 0.009*"blood" + '
  '0.008*"date" + 0.007*"discharg" + 0.007*"histori" + 0.006*"pain" + '
  '0.006*"follow" + 0.006*"time"'),
 (2,
  '0.011*"procedur" + 0.008*"patient" + 0.007*"note" + 0.007*"numer" + '
  '0.007*"arteri" + 0.006*"leav" + 0.006*"interpret" + 0.006*"sedat" + '
  '0.006*"place" + 0.006*"identifi"'),
 (3,
  '0.018*"plan" + 0.016*"respons" + 0.014*"action" + 0.013*"pain" + '
  '0.013*"assess" + 0.009*"patient" + 0.009*"continu" + 0.009*"monitor" + '
  '0.007*"give" + 0.007*"remain"'),
 (4,
  '0.008*"lab" + 0.008*"patient" + 0.008*"continu" + 0.007*"start" + '
  '0.006*"hour" + 0.005*"assess" + 0.005*"follow" + 0.005*"statu" + '
  '0.005*"respiratori" + 0.005*"lymphoma"'),
 (5,
  '0.028*"assess" + 0.018*"lung" + 0.017*"continu" + 0.0

In [197]:
# Visualize the topics and words
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
#vis

In [195]:
#train LDA model using TFIDF corpus, chunk size is 5000 documents, lda is updated after every chunk size, 2 full passes through the corpus for training, produce 10 topics
lda_model2 = gensim.models.ldamodel.LdaModel(corpus=corpus_tfidf,
                                           id2word=dictionary,
                                           num_topics=2, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=5000,
                                           passes=2)

In [22]:
# Print the Keywords in the 10 topics using the TFIDF corpus
pprint(lda_model2.print_topics())
doc_lda2 = lda_model2[corpus_tfidf]

[(0,
  '0.019*"action" + 0.016*"respons" + 0.010*"plan" + 0.009*"assess" + '
  '0.007*"pain" + 0.006*"continu" + 0.006*"monitor" + 0.005*"failur" + '
  '0.005*"acut" + 0.005*"cont"'),
 (1,
  '0.020*"cvvh" + 0.017*"nutrit" + 0.016*"weight" + 0.015*"ioniz" + '
  '0.013*"diet" + 0.012*"protein" + 0.011*"arteri" + 0.011*"calcium" + '
  '0.011*"kcal" + 0.011*"recommend"'),
 (2,
  '0.019*"valv" + 0.012*"aortic" + 0.012*"contrast" + 0.011*"mitral" + '
  '0.011*"normal" + 0.010*"leaflet" + 0.008*"wall" + 0.008*"doppler" + '
  '0.007*"ventricular" + 0.007*"dilat"'),
 (3,
  '0.008*"puls" + 0.007*"balanc" + 0.007*"total" + 0.007*"hour" + 0.006*"code" '
  '+ 0.006*"lab" + 0.006*"assess" + 0.005*"prophylaxi" + 0.005*"respiratori" + '
  '0.005*"line"'),
 (4,
  '0.071*"trace" + 0.044*"wave" + 0.039*"previou" + 0.032*"sinu" + '
  '0.030*"compar" + 0.026*"rhythm" + 0.024*"lead" + 0.020*"specif" + '
  '0.020*"diagnost" + 0.020*"atrial"'),
 (5,
  '0.014*"contrast" + 0.013*"fractur" + 0.010*"hemorrhag" + 

In [2]:
# Visualize the topics and words based on TFIDF weight corpus
pyLDAvis.enable_notebook()
vis2 = pyLDAvis.gensim.prepare(lda_model2, corpus_tfidf, dictionary)
vis2

In [ ]:
#the following steps are being used to validate the LDA model using Diabetes patients and suicide attempt patients

In [16]:
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
from datetime import datetime
dateparse = lambda dates: [pd.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dates]

In [19]:
diagnosis_icd9 = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/DIAGNOSES_ICD.csv", usecols= ["SUBJECT_ID", "ICD9_CODE"])
diag_name = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/D_ICD_DIAGNOSES.csv", usecols= ["ICD9_CODE", "LONG_TITLE"])
code_description = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/DRGCODES.csv")
patients = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/PATIENTS.csv", parse_dates= ["DOB"], date_parser=dateparse, usecols=["SUBJECT_ID", "DOB"])
admissions = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/ADMISSIONS.csv", parse_dates= ["ADMITTIME"], date_parser=dateparse, usecols=["SUBJECT_ID", "ADMITTIME"])
notes = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/NOTEEVENTS.csv", usecols = ['SUBJECT_ID','CATEGORY', 'TEXT'])

/global/homes/d/dsmorrow/.conda/envs/intern/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """


In [18]:
#filter for only diabetes ICD9 codes
diag_db_DF = diagnosis_icd9[diagnosis_icd9["ICD9_CODE"].str.startswith(('159','167','181','226','227','644'), na = False)]
diag_db_DF.head()

,SUBJECT_ID,ICD9_CODE
6161,657,2270
13835,1343,226
25322,3193,64421
26877,3357,2270
30270,2159,2270


In [20]:
#merge diabetes dataframe with notes
icd9_db_notes = pd.merge(diag_db_DF, notes, on ='SUBJECT_ID')
icd9_db_notes

,SUBJECT_ID,ICD9_CODE,CATEGORY,TEXT
0,657,2270,Discharge summary,Admission Date: [**2124-9-6**] Discharge ...
1,657,2270,Discharge summary,Admission Date: [**2124-9-6**] Discharge ...
2,657,2270,Echo,PATIENT/TEST INFORMATION:\nIndication: HTN\nHe...
3,657,2270,ECG,Sinus rhythm\n- supraventricular extrasystoles...
4,657,2270,Radiology,PERSANTINE MIBI ...
...,...,...,...,...
9683,97842,2270,Radiology,[**2144-4-15**] 1:00 PM\n CT ABD W&W/O C ...
9684,97842,2270,Radiology,[**2144-4-13**] 2:35 PM\n CT HEAD W/O CONTRAST...
9685,97842,2270,Radiology,[**2144-4-13**] 2:24 PM\n CHEST (PORTABLE AP) ...
9686,97842,2270,Radiology,[**2144-3-25**] 9:41 AM\n CTA CHEST W&W/O C&RE...


In [22]:
icd9_db_notes.drop_duplicates()

,SUBJECT_ID,ICD9_CODE,CATEGORY,TEXT
0,657,2270,Discharge summary,Admission Date: [**2124-9-6**] Discharge ...
1,657,2270,Discharge summary,Admission Date: [**2124-9-6**] Discharge ...
2,657,2270,Echo,PATIENT/TEST INFORMATION:\nIndication: HTN\nHe...
3,657,2270,ECG,Sinus rhythm\n- supraventricular extrasystoles...
4,657,2270,Radiology,PERSANTINE MIBI ...
...,...,...,...,...
9531,97842,2270,Radiology,[**2144-4-15**] 1:00 PM\n CT ABD W&W/O C ...
9532,97842,2270,Radiology,[**2144-4-13**] 2:35 PM\n CT HEAD W/O CONTRAST...
9533,97842,2270,Radiology,[**2144-4-13**] 2:24 PM\n CHEST (PORTABLE AP) ...
9534,97842,2270,Radiology,[**2144-3-25**] 9:41 AM\n CTA CHEST W&W/O C&RE...


In [26]:
# Load the regular expression library
import re

In [28]:
# Remove punctuation and anything that isnt a character or number 
icd9_db_notes['text_processed'] = icd9_db_notes['TEXT'].map(lambda x: re.sub('[_\\W]+', ' ', x))

In [29]:
# Convert to lowercase
icd9_db_notes['text_processed'] = icd9_db_notes['text_processed'].map(lambda x: x.lower())

In [37]:
#remove numbers
icd9_db_notes['text_processed'] = icd9_db_notes['text_processed'].map(lambda x: re.sub('(\\s\\d+)', ' ', x))

In [38]:
# Print out the first rows of papers to ensure re working properly
icd9_db_notes

,SUBJECT_ID,ICD9_CODE,CATEGORY,TEXT,text_processed,index
0,657,2270,Discharge summary,Admission Date: [**2124-9-6**] Discharge ...,admission date discharge date date of bi...,0
1,657,2270,Discharge summary,Admission Date: [**2124-9-6**] Discharge ...,admission date discharge date date of bi...,1
2,657,2270,Echo,PATIENT/TEST INFORMATION:\nIndication: HTN\nHe...,patient test information indication htn height...,2
3,657,2270,ECG,Sinus rhythm\n- supraventricular extrasystoles...,sinus rhythm supraventricular extrasystoles la...,3
4,657,2270,Radiology,PERSANTINE MIBI ...,persantine mibi clip clip number radiology re...,4
...,...,...,...,...,...,...
9683,97842,2270,Radiology,[**2144-4-15**] 1:00 PM\n CT ABD W&W/O C ...,pm ct abd w w o c clip clip number radio...,9683
9684,97842,2270,Radiology,[**2144-4-13**] 2:35 PM\n CT HEAD W/O CONTRAST...,pm ct head w o contrast clip clip number...,9684
9685,97842,2270,Radiology,[**2144-4-13**] 2:24 PM\n CHEST (PORTABLE AP) ...,pm chest portable ap clip clip number ra...,9685
9686,97842,2270,Radiology,[**2144-3-25**] 9:41 AM\n CTA CHEST W&W/O C&RE...,am cta chest w w o c recons non coronary...,9686


In [39]:
#adding additional index column for calling
index = tuple(range(0, 9688, 1))
icd9_db_notes["index"] = index

In [185]:
icd9_db_notes = icd9_db_notes.drop_duplicates()

In [186]:
diabetes_text = icd9_db_notes[['text_processed','index']]
diabetes_text

,text_processed,index
0,admission date discharge date date of bi...,0
1,admission date discharge date date of bi...,1
2,patient test information indication htn height...,2
3,sinus rhythm supraventricular extrasystoles la...,3
4,persantine mibi clip clip number radiology re...,4
...,...,...
9683,pm ct abd w w o c clip clip number radio...,9683
9684,pm ct head w o contrast clip clip number...,9684
9685,pm chest portable ap clip clip number ra...,9685
9686,am cta chest w w o c recons non coronary...,9686


In [74]:
d_documents = diabetes_text

In [44]:
#function to perform lemmatize and stem preprocessing steps on the data set.
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [45]:
#use preprocess function 
d_proc_docs = d_documents['text_processed'].map(preprocess)

In [50]:
#create dictionary of words and number of appearances 
d_dictionary = gensim.corpora.Dictionary(d_proc_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abdomin
1 adenoma
2 admiss
3 adren
4 adrenalectomi
5 advis
6 afebril
7 area
8 benign
9 biopsi
10 birth


In [51]:
#filter out words that appear in less than (15) documents
#only keep the first 10000
d_dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [52]:
#go through each document and report words and occurrences using doc2box for token id and amount
d_bow_corpus = [d_dictionary.doc2bow(doc) for doc in d_proc_docs]

In [53]:
## determine which topic a certain document belongs to
#765 is a sample index from the diabetes group
for index, score in sorted(lda_model[d_bow_corpus[765]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.3928428292274475	 
Topic: 0.017*"tablet" + 0.015*"daili" + 0.015*"patient" + 0.012*"blood" + 0.011*"histori" + 0.011*"discharg" + 0.011*"pain" + 0.008*"date" + 0.008*"time" + 0.007*"home" 
Score: 0.3397396206855774	 
Topic: 0.013*"continu" + 0.013*"blood" + 0.013*"cultur" + 0.012*"patient" + 0.012*"renal" + 0.012*"like" + 0.009*"failur" + 0.008*"fever" + 0.008*"neg" + 0.008*"urin" 
Score: 0.1051432266831398	 
Topic: 0.021*"assess" + 0.012*"hour" + 0.011*"respiratori" + 0.011*"comment" + 0.010*"total" + 0.009*"puls" + 0.009*"sound" + 0.009*"balanc" + 0.009*"code" + 0.008*"line" 
Score: 0.06217994540929794	 
Topic: 0.026*"contrast" + 0.012*"abdomen" + 0.012*"clip" + 0.011*"small" + 0.010*"right" + 0.010*"pelvi" + 0.009*"reason" + 0.009*"leav" + 0.009*"liver" + 0.009*"fluid" 
Score: 0.05533013120293617	 
Topic: 0.035*"right" + 0.034*"leav" + 0.033*"fractur" + 0.021*"vein" + 0.018*"reason" + 0.017*"clip" + 0.013*"year" + 0.012*"report" + 0.012*"arteri" + 0.011*"extrem" 
Score: 0.0

In [57]:
#this is the note for index 765
icd9_db_notes['TEXT'].iloc[765]

'[**2163-9-21**] 4:45 PM\n CT HEAD W/O CONTRAST                                            Clip # [**Clip Number (Radiology) 41032**]\n Reason: please evaluate skull base w/2.5mm thin slices, using faduci\n Admitting Diagnosis: PITUITARY TUMOR\n ______________________________________________________________________________\n [**Hospital 4**] MEDICAL CONDITION:\n  54 year old woman with\n REASON FOR THIS EXAMINATION:\n  please evaluate skull base w/2.5mm thin slices, using faducials.\n No contraindications for IV contrast\n ______________________________________________________________________________\n                                 FINAL REPORT\n CT HEAD WITHOUT CONTRAST, [**2163-9-21**]\n\n HISTORY:  Pituitary adenoma.  Presurgical planning.\n\n Contiguous axial images were obtained through the brain.  No contrast was\n administered.  Comparison to a head CT of [**2163-9-20**].  0.625-mm\n high-resolution images were reconstructed through the skull base.\n\n FINDINGS:  There have be

In [68]:
#what subject id belongs to this index?
icd9_db_notes.iloc[765]

SUBJECT_ID                                                     8795
ICD9_CODE                                                      2273
CATEGORY                                                  Radiology
TEXT              [**2163-9-21**] 4:45 PM\n CT HEAD W/O CONTRAST...
text_processed          pm ct head w o contrast clip clip number...
index                                                           765
Name: 765, dtype: object

In [187]:
icd9_db_notes.loc[icd9_db_notes['SUBJECT_ID'] == 8795]

,SUBJECT_ID,ICD9_CODE,CATEGORY,TEXT,text_processed,index
716,8795,2273,Discharge summary,Admission Date: [**2157-2-7**] Discharg...,admission date discharge date date of bi...,716
717,8795,2273,Discharge summary,Admission Date: [**2163-9-15**] ...,admission date discharge date date of bi...,717
718,8795,2273,Echo,PATIENT/TEST INFORMATION:\nIndication: H/O car...,patient test information indication h o cardia...,718
719,8795,2273,Echo,PATIENT/TEST INFORMATION:\nIndication: H/O car...,patient test information indication h o cardia...,719
720,8795,2273,Echo,PATIENT/TEST INFORMATION:\nIndication: H/O MVR...,patient test information indication h o mvr tv...,720
...,...,...,...,...,...,...
803,8795,2273,Nursing/other,CV: V-paced. No underlying rhythm. EP in to ...,cv v paced no underlying rhythm ep in to see p...,803
804,8795,2273,Nursing/other,"CV: V-paced, underlying afib vs asystole. Un...",cv v paced underlying afib vs asystole unable ...,804
805,8795,2273,Nursing/other,Respiratory Care\nFollowed overnight with Albu...,respiratory care followed overnight with albut...,805
806,8795,2273,Nursing/other,Neuro: pt awake alert oriented. Has more enege...,neuro pt awake alert oriented has more enegery...,806


In [173]:
## determine which topic a certain document belongs to
#716 is  index from the diabetes group - discharge note for subject 8795
for index, score in sorted(lda_model[d_bow_corpus[716]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.8064509630203247	 
Topic: 0.014*"patient" + 0.012*"tablet" + 0.009*"daili" + 0.009*"blood" + 0.008*"date" + 0.007*"discharg" + 0.007*"histori" + 0.006*"pain" + 0.006*"follow" + 0.006*"time" 
Score: 0.19244776666164398	 
Topic: 0.034*"normal" + 0.029*"valv" + 0.024*"leav" + 0.020*"aortic" + 0.019*"ventricular" + 0.014*"right" + 0.014*"mitral" + 0.014*"leaflet" + 0.012*"size" + 0.011*"wall" 


In [54]:
## determine which topic a certain document belongs to
#92 is a sample index from the diabetes group
for index, score in sorted(lda_model[d_bow_corpus[92]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.7348766922950745	 
Topic: 0.017*"tablet" + 0.015*"daili" + 0.015*"patient" + 0.012*"blood" + 0.011*"histori" + 0.011*"discharg" + 0.011*"pain" + 0.008*"date" + 0.008*"time" + 0.007*"home" 
Score: 0.1791783571243286	 
Topic: 0.013*"continu" + 0.013*"blood" + 0.013*"cultur" + 0.012*"patient" + 0.012*"renal" + 0.012*"like" + 0.009*"failur" + 0.008*"fever" + 0.008*"neg" + 0.008*"urin" 
Score: 0.04973447322845459	 
Topic: 0.037*"valv" + 0.037*"normal" + 0.028*"leav" + 0.027*"aortic" + 0.023*"ventricular" + 0.020*"mitral" + 0.018*"trace" + 0.016*"leaflet" + 0.015*"right" + 0.014*"wall" 
Score: 0.02601848542690277	 
Topic: 0.026*"procedur" + 0.019*"cathet" + 0.017*"patient" + 0.015*"place" + 0.014*"identifi" + 0.014*"namepattern" + 0.013*"numer" + 0.013*"tube" + 0.013*"picc" + 0.013*"right" 


In [58]:
#this is the actual note for index 92
icd9_db_notes['TEXT'].iloc[92]

"SMICU nsg progress note\n52yo women with hx hypertension s/p adrenalectomy [**2112-1-15**] admitted to smicu with hypoxia. See fhpa for details pmh/hpi.\nR.O.S.\n[**Name (NI) **] Pt in resp distress on admit with sats 80's on 100%nr +6l nc. Pt placed on bi-pap with improvement in 02 sats to 90-93% with abg 63/44/7.41. C-xray with significant ?infiltrate on lt. Pt only able to tolerate bipap for about 1.5 hrs. Now on 100%nr with np. Last abg 54/40/7.47. Possible intub discuss with pt. Will try to avoid intub. Will try bipab again.\nCardiac- Bp and hr stable. Pt denies cp. Cont with good uo after lasix given on 12r. Will cont to follow\n[**Name (NI) **] Pt denies pain. Abdomen soft non-tender with good bowel  sounds. Pt npo except meds.\nPlan- Follow resp status closely. Will most likely have to intub. Give small doses ativan to help with agitation from hypoxia. Antibiotics as ordered. Follow fs q 6hrs.\n"

In [182]:
#what subject id belongs to 92?
icd9_db_notes.iloc[92]

SUBJECT_ID                                                     2159
ICD9_CODE                                                      2270
CATEGORY                                              Nursing/other
TEXT              SMICU nsg progress note\n52yo women with hx hy...
text_processed    smicu nsg progress note yo women with hx hyper...
index                                                            92
Name: 92, dtype: object

In [188]:
#all notes for subject 2159
icd9_db_notes.loc[icd9_db_notes['SUBJECT_ID'] == 2159][0:5]

,SUBJECT_ID,ICD9_CODE,CATEGORY,TEXT,text_processed,index
69,2159,2270,Discharge summary,Admission Date: [**2112-1-14**] Discharge...,admission date discharge date date of bi...,69
70,2159,2270,Discharge summary,Admission Date: [**2112-1-14**] Discharge...,admission date discharge date date of bi...,70
71,2159,2270,ECG,Sinus rhythm. Low QRS voltage in the limb lead...,sinus rhythm low qrs voltage in the limb leads...,71
72,2159,2270,ECG,Sinus rhythm with physiologic sinus arrhythmia...,sinus rhythm with physiologic sinus arrhythmia...,72
73,2159,2270,Radiology,[**2112-1-19**] 8:21 AM\n CHEST (PORTABLE AP) ...,am chest portable ap clip clip number ra...,73


In [184]:
## determine which topic a certain document belongs to
#69 is index from the subject 2159 - discharge note
for index, score in sorted(lda_model[d_bow_corpus[69]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.9447231888771057	 
Topic: 0.014*"patient" + 0.012*"tablet" + 0.009*"daili" + 0.009*"blood" + 0.008*"date" + 0.007*"discharg" + 0.007*"histori" + 0.006*"pain" + 0.006*"follow" + 0.006*"time" 
Score: 0.0533100850880146	 
Topic: 0.026*"chest" + 0.023*"reason" + 0.021*"right" + 0.019*"clip" + 0.015*"leav" + 0.013*"tube" + 0.012*"year" + 0.012*"effus" + 0.012*"examin" + 0.011*"report" 


In [61]:
## determine which topic a certain document belongs to
#305 is a sample index from the diabetes
for index, score in sorted(lda_model[d_bow_corpus[306]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.5050062537193298	 
Topic: 0.017*"tablet" + 0.015*"daili" + 0.015*"patient" + 0.012*"blood" + 0.011*"histori" + 0.011*"discharg" + 0.011*"pain" + 0.008*"date" + 0.008*"time" + 0.007*"home" 
Score: 0.26892733573913574	 
Topic: 0.013*"continu" + 0.013*"blood" + 0.013*"cultur" + 0.012*"patient" + 0.012*"renal" + 0.012*"like" + 0.009*"failur" + 0.008*"fever" + 0.008*"neg" + 0.008*"urin" 
Score: 0.1312897652387619	 
Topic: 0.021*"head" + 0.019*"contrast" + 0.018*"hemorrhag" + 0.018*"leav" + 0.017*"right" + 0.013*"clip" + 0.011*"reason" + 0.011*"mass" + 0.010*"imag" + 0.010*"brain" 
Score: 0.04904064163565636	 
Topic: 0.026*"procedur" + 0.019*"cathet" + 0.017*"patient" + 0.015*"place" + 0.014*"identifi" + 0.014*"namepattern" + 0.013*"numer" + 0.013*"tube" + 0.013*"picc" + 0.013*"right" 
Score: 0.03180340677499771	 
Topic: 0.035*"right" + 0.034*"leav" + 0.033*"fractur" + 0.021*"vein" + 0.018*"reason" + 0.017*"clip" + 0.013*"year" + 0.012*"report" + 0.012*"arteri" + 0.011*"extrem" 
Sco

In [62]:
icd9_db_notes['TEXT'].iloc[306]

'[**2118-6-13**] 7:44 AM\n CT ABDOMEN W/CONTRAST; CT PELVIS W/CONTRAST                     Clip # [**Clip Number (Radiology) 1369**]\n Reason: dx: status post surgery, abd pain\n Field of view: 43 Contrast: OPTIRAY Amt: 130\n ______________________________________________________________________________\n [**Hospital 2**] MEDICAL CONDITION:\n  49 year old man with\n REASON FOR THIS EXAMINATION:\n  dx: status post surgery, abd pain\n ______________________________________________________________________________\n                                 FINAL REPORT\n INDICATION:  49-year-old man with history of recent right adrenal hemorrhage\n status post arterial embolization.  Continued abdominal pain.\n\n COMPARISON:  [**2118-4-26**] and [**2115-11-6**].\n\n TECHNIQUE:  MDCT axial images of the abdomen and pelvis were obtained\n following the administration of IV Optiray contrast with multiplanar images\n reformatted.\n\n CT ABDOMEN WITH IV CONTRAST:  There are four sub-5-mm pulmonary nodul

In [ ]:
#----------------------------------------------------------------
#need for validation
#looking only at suicide attempts now
#based on diabetes results; should we be looking at the number of topics that the note matches or the probability and install a threshold #?

In [101]:
#filter for only suicide attempts ICD9 codes
diag_suicide_DF = diagnosis_icd9[diagnosis_icd9["ICD9_CODE"].str.startswith(('E95'), na = False)]
diag_suicide_DF.head()

,SUBJECT_ID,ICD9_CODE
102,119,E956
350,71,E9503
760,22,E9502
761,22,E9503
2036,303,E9500


In [102]:
#merge suicide dataframe with notes
icd9_suicide = pd.merge(diag_suicide_DF, notes, on ='SUBJECT_ID')
icd9_suicide

,SUBJECT_ID,ICD9_CODE,CATEGORY,TEXT
0,119,E956,ECG,Sinus rhythm\nHigh QRS voltage - probable norm...
1,119,E956,ECG,"Normal sinus rhythm, rate 94\nNonspecific ante..."
2,119,E956,Radiology,[**2159-9-12**] 6:44 AM\n WRIST(3 + VIEWS) LEF...
3,119,E956,Radiology,[**2159-9-12**] 6:46 AM\n CHEST (PRE-OP AP ONL...
4,119,E956,Nursing/other,"awake,alert, and oriented x3. pt is calm and c..."
...,...,...,...,...
23020,97625,E9500,Radiology,[**2105-2-17**] 7:17 AM\n CHEST (PORTABLE AP) ...
23021,97625,E9500,Radiology,[**2105-2-16**] 12:08 AM\n CHEST (PORTABLE AP)...
23022,97625,E9500,Physician,TITLE:\n Chief Complaint: acute hepatic inj...
23023,97625,E9500,Physician,TITLE:\n Chief Complaint: acute hepatic inj...


In [104]:
#bring in the admittime column from separate csv
icd9_suicide = pd.merge(icd9_suicide, admissions, on="SUBJECT_ID")
icd9_suicide.head()

,SUBJECT_ID,ICD9_CODE,CATEGORY,TEXT,ADMITTIME
0,119,E956,ECG,Sinus rhythm\nHigh QRS voltage - probable norm...,2159-09-12 15:26:00
1,119,E956,ECG,"Normal sinus rhythm, rate 94\nNonspecific ante...",2159-09-12 15:26:00
2,119,E956,Radiology,[**2159-9-12**] 6:44 AM\n WRIST(3 + VIEWS) LEF...,2159-09-12 15:26:00
3,119,E956,Radiology,[**2159-9-12**] 6:46 AM\n CHEST (PRE-OP AP ONL...,2159-09-12 15:26:00
4,119,E956,Nursing/other,"awake,alert, and oriented x3. pt is calm and c...",2159-09-12 15:26:00


In [111]:
icd9_suicide_notes = icd9_suicide[['SUBJECT_ID', 'ICD9_CODE', "ADMITTIME"]]

In [112]:
icd9_suicide_notes

,SUBJECT_ID,ICD9_CODE,ADMITTIME
0,119,E956,2159-09-12 15:26:00
1,119,E956,2159-09-12 15:26:00
2,119,E956,2159-09-12 15:26:00
3,119,E956,2159-09-12 15:26:00
4,119,E956,2159-09-12 15:26:00
...,...,...,...
42548,97625,E9500,2105-02-15 18:00:00
42549,97625,E9500,2105-02-15 18:00:00
42550,97625,E9500,2105-02-15 18:00:00
42551,97625,E9500,2105-02-15 18:00:00


In [113]:
#need to remove duplicates
icd9_suicide_notes_ND = icd9_suicide_notes.drop_duplicates()
icd9_suicide_notes_ND

,SUBJECT_ID,ICD9_CODE,ADMITTIME
0,119,E956,2159-09-12 15:26:00
9,71,E9503,2164-02-03 22:07:00
22,22,E9502,2196-04-09 12:26:00
32,22,E9503,2196-04-09 12:26:00
42,303,E9500,2160-03-17 18:37:00
...,...,...,...
42488,98177,E9588,2145-08-11 02:36:00
42491,99868,E9500,2142-09-07 05:36:00
42494,99868,E9503,2142-09-07 05:36:00
42497,99868,E9509,2142-09-07 05:36:00


In [114]:
#group by subject id and then icd9, count number of admissions
icd9_suicide_notes_ND['count'] = icd9_suicide_notes_ND.groupby(by=['SUBJECT_ID','ICD9_CODE'])['ADMITTIME'].transform('count')
icd9_suicide_notes_ND

/global/homes/d/dsmorrow/.conda/envs/intern/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,SUBJECT_ID,ICD9_CODE,ADMITTIME,count
0,119,E956,2159-09-12 15:26:00,1
9,71,E9503,2164-02-03 22:07:00,1
22,22,E9502,2196-04-09 12:26:00,1
32,22,E9503,2196-04-09 12:26:00,1
42,303,E9500,2160-03-17 18:37:00,2
...,...,...,...,...
42488,98177,E9588,2145-08-11 02:36:00,1
42491,99868,E9500,2142-09-07 05:36:00,1
42494,99868,E9503,2142-09-07 05:36:00,1
42497,99868,E9509,2142-09-07 05:36:00,1


In [116]:
#only looking at patients that were admitted more than once
icd9_suicide_notes = icd9_suicide_notes_ND.loc[icd9_suicide_notes_ND['count'] >1 ]
icd9_suicide_notes

,SUBJECT_ID,ICD9_CODE,ADMITTIME,count
42,303,E9500,2160-03-17 18:37:00,2
43,303,E9500,2163-03-29 23:04:00,2
142,523,E9509,2137-06-20 03:28:00,2
143,523,E9509,2141-12-19 19:59:00,2
329,1552,E9503,2195-01-30 23:04:00,2
...,...,...,...,...
41184,91465,E9503,2186-08-22 08:28:00,4
41185,91465,E9503,2186-09-06 14:45:00,4
41804,95136,E9500,2134-03-20 17:58:00,3
41805,95136,E9500,2137-04-02 18:24:00,3


In [124]:
#need to merge back in the text and category columns
icd9_suicide2 = icd9_suicide[['SUBJECT_ID','CATEGORY', 'TEXT']]

In [127]:
#merge back in the text and category columns
icd9_suicide_notes2 = pd.merge(icd9_suicide_notes, icd9_suicide2, on="SUBJECT_ID")

In [129]:
# Remove punctuation and anything that isnt a character or number 
icd9_suicide_notes2['text_processed'] = icd9_suicide_notes2['TEXT'].map(lambda x: re.sub('[_\\W]+', ' ', x))

In [130]:
# Convert to lowercase
icd9_suicide_notes2['text_processed'] = icd9_suicide_notes2['text_processed'].map(lambda x: x.lower())

In [131]:
#remove numbers
icd9_suicide_notes2['text_processed'] = icd9_suicide_notes2['text_processed'].map(lambda x: re.sub('(\\s\\d+)', ' ', x))

In [132]:
icd9_suicide_notes2

,SUBJECT_ID,ICD9_CODE,ADMITTIME,count,CATEGORY,TEXT,text_processed
0,303,E9500,2160-03-17 18:37:00,2,Discharge summary,Admission Date: [**2160-3-17**] Discharge...,admission date discharge date date of bi...
1,303,E9500,2160-03-17 18:37:00,2,Discharge summary,Admission Date: [**2160-3-17**] Discharge...,admission date discharge date date of bi...
2,303,E9500,2160-03-17 18:37:00,2,Discharge summary,Admission Date: [**2163-3-29**] ...,admission date discharge date date of bi...
3,303,E9500,2160-03-17 18:37:00,2,Discharge summary,Admission Date: [**2163-3-29**] ...,admission date discharge date date of bi...
4,303,E9500,2160-03-17 18:37:00,2,ECG,Sinus rhythm\nNormal ECG for age\n\n,sinus rhythm normal ecg for age
...,...,...,...,...,...,...,...
201589,95136,E9500,2137-04-22 20:24:00,3,Radiology,[**2137-10-21**] 1:39 PM\n CHEST (PA & LAT) ...,pm chest pa lat clip clip number radiolo...
201590,95136,E9500,2137-04-22 20:24:00,3,Radiology,[**2137-10-21**] 1:39 PM\n CHEST (PA & LAT) ...,pm chest pa lat clip clip number radiolo...
201591,95136,E9500,2137-04-22 20:24:00,3,Radiology,FDG TUMOR IMAGING (PET-CT) ...,fdg tumor imaging pet ct clip clip number radi...
201592,95136,E9500,2137-04-22 20:24:00,3,Radiology,FDG TUMOR IMAGING (PET-CT) ...,fdg tumor imaging pet ct clip clip number radi...


In [135]:
icd9_suicide_notes2 = icd9_suicide_notes2.drop_duplicates()

In [136]:
#adding additional index column for calling
index = tuple(range(0, 25431, 1))
icd9_suicide_notes2["index"] = index

In [137]:
suicide_text = icd9_suicide_notes2[['text_processed','index']]
suicide_text

,text_processed,index
0,admission date discharge date date of bi...,0
2,admission date discharge date date of bi...,1
4,sinus rhythm normal ecg for age,2
6,sinus rhythm slight non specific inferior st t...,3
8,pm trauma series lat c spine ap cxr ap p...,4
...,...,...
201579,fdg tumor imaging pet ct clip clip number radi...,25426
201582,pm ct head w o contrast clip reason ev...,25427
201585,am ct head w o contrast clip reason ne...,25428
201588,pm chest pa lat clip clip number radiolo...,25429


In [138]:
suicide_documents = suicide_text

In [139]:
#use preprocess function on suicide group
suicide_proc_docs = suicide_documents['text_processed'].map(preprocess)

In [140]:
#create dictionary of words and number of appearances 
suicide_dictionary = gensim.corpora.Dictionary(suicide_proc_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abdomin
1 adenoma
2 admiss
3 adren
4 adrenalectomi
5 advis
6 afebril
7 area
8 benign
9 biopsi
10 birth


In [141]:
#filter out words that appear in less than (15) documents
#only keep the first 10000
suicide_dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [162]:
#go through each document and report words and occurrences using doc2box for token id and amount
suicide_bow_corpus = [suicide_dictionary.doc2bow(doc) for doc in suicide_proc_docs]

In [163]:
## determine which topic a certain document belongs to
#100 is a sample index from the suicide group - this is a nursing note
for index, score in sorted(lda_model[suicide_bow_corpus[100]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.7652101516723633	 
Topic: 0.014*"patient" + 0.012*"tablet" + 0.009*"daili" + 0.009*"blood" + 0.008*"date" + 0.007*"discharg" + 0.007*"histori" + 0.006*"pain" + 0.006*"follow" + 0.006*"time" 
Score: 0.11040948331356049	 
Topic: 0.018*"plan" + 0.016*"respons" + 0.014*"action" + 0.013*"pain" + 0.013*"assess" + 0.009*"patient" + 0.009*"continu" + 0.009*"monitor" + 0.007*"give" + 0.007*"remain" 
Score: 0.10544098913669586	 
Topic: 0.034*"normal" + 0.029*"valv" + 0.024*"leav" + 0.020*"aortic" + 0.019*"ventricular" + 0.014*"right" + 0.014*"mitral" + 0.014*"leaflet" + 0.012*"size" + 0.011*"wall" 


In [191]:
#what is this note?
icd9_suicide_notes2['TEXT'].iloc[100]

'MICU NURSING PROGRESS NOTE 0700-1500\nADDENDUM TO NOTE,\n\nBLOOD CX X2 AND URINE C&S/UA SENT.\nR FEMORAL DIALYSIS LINE D/CED WITHOUT INCIDENT. MANUAL PRESSURE APPLIED FOR 15 BUT REBLED. MANUAL PRESSURE APPLIED FOR ADDITIONAL 15 MIN WITH GOOD HEMASTASIS. DSD APPLIED. NO OOZE . SMALL QUARTER SIZE HEMATOMA PRESENT PRIOR TO D/C OF LINE. IT HAS NOT CHANGED IN SIZE AND IS SUPERIOR AND MEDIAL TO INSERTION SITE.\n'

In [164]:
#subject id?
icd9_suicide_notes2.iloc[100]

SUBJECT_ID                                                      523
ICD9_CODE                                                     E9509
ADMITTIME                                       2137-06-20 03:28:00
count                                                             2
CATEGORY                                              Nursing/other
TEXT              MICU NURSING PROGRESS NOTE 0700-1500\nADDENDUM...
text_processed    micu nursing progress note   addendum to note ...
index                                                           100
Name: 200, dtype: object

In [165]:
#all notes for subject 523
icd9_suicide_notes2.loc[icd9_suicide_notes2['SUBJECT_ID'] == 523][0:5]

,SUBJECT_ID,ICD9_CODE,ADMITTIME,count,CATEGORY,TEXT,text_processed,index
120,523,E9509,2137-06-20 03:28:00,2,Discharge summary,Admission Date: [**2137-6-20**] ...,admission date discharge date date of bi...,60
122,523,E9509,2137-06-20 03:28:00,2,Discharge summary,Admission Date: [**2141-12-19**] ...,admission date discharge date date of bi...,61
124,523,E9509,2137-06-20 03:28:00,2,Echo,PATIENT/TEST INFORMATION:\nIndication: Hyperte...,patient test information indication hypertensi...,62
126,523,E9509,2137-06-20 03:28:00,2,ECG,Artifact is present. Sinus rhythm. Normal tr...,artifact is present sinus rhythm normal tracin...,63
128,523,E9509,2137-06-20 03:28:00,2,ECG,Sinus arrhythmia. Tall peaked P waves and rig...,sinus arrhythmia tall peaked p waves and right...,64


In [169]:
## determine which topic a certain document belongs to
#60 is index from the suicide group - this is a discharge note specifically for subject 523
for index, score in sorted(lda_model[suicide_bow_corpus[60]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.34231090545654297	 
Topic: 0.022*"contrast" + 0.018*"right" + 0.017*"clip" + 0.016*"leav" + 0.014*"reason" + 0.009*"report" + 0.009*"imag" + 0.009*"mass" + 0.009*"year" + 0.009*"radiolog" 
Score: 0.2560170590877533	 
Topic: 0.018*"plan" + 0.016*"respons" + 0.014*"action" + 0.013*"pain" + 0.013*"assess" + 0.009*"patient" + 0.009*"continu" + 0.009*"monitor" + 0.007*"give" + 0.007*"remain" 
Score: 0.18672949075698853	 
Topic: 0.014*"patient" + 0.012*"tablet" + 0.009*"daili" + 0.009*"blood" + 0.008*"date" + 0.007*"discharg" + 0.007*"histori" + 0.006*"pain" + 0.006*"follow" + 0.006*"time" 
Score: 0.09038311243057251	 
Topic: 0.026*"chest" + 0.023*"reason" + 0.021*"right" + 0.019*"clip" + 0.015*"leav" + 0.013*"tube" + 0.012*"year" + 0.012*"effus" + 0.012*"examin" + 0.011*"report" 
Score: 0.08346293866634369	 
Topic: 0.034*"normal" + 0.029*"valv" + 0.024*"leav" + 0.020*"aortic" + 0.019*"ventricular" + 0.014*"right" + 0.014*"mitral" + 0.014*"leaflet" + 0.012*"size" + 0.011*"wall" 
Sco

In [170]:
## determine which topic a certain document belongs to
#61 is index from the suicide group - this is a discharge note specifically for subject 523
for index, score in sorted(lda_model[suicide_bow_corpus[61]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.45882004499435425	 
Topic: 0.014*"patient" + 0.012*"tablet" + 0.009*"daili" + 0.009*"blood" + 0.008*"date" + 0.007*"discharg" + 0.007*"histori" + 0.006*"pain" + 0.006*"follow" + 0.006*"time" 
Score: 0.17024625837802887	 
Topic: 0.022*"contrast" + 0.018*"right" + 0.017*"clip" + 0.016*"leav" + 0.014*"reason" + 0.009*"report" + 0.009*"imag" + 0.009*"mass" + 0.009*"year" + 0.009*"radiolog" 
Score: 0.13856647908687592	 
Topic: 0.018*"plan" + 0.016*"respons" + 0.014*"action" + 0.013*"pain" + 0.013*"assess" + 0.009*"patient" + 0.009*"continu" + 0.009*"monitor" + 0.007*"give" + 0.007*"remain" 
Score: 0.10936947166919708	 
Topic: 0.034*"normal" + 0.029*"valv" + 0.024*"leav" + 0.020*"aortic" + 0.019*"ventricular" + 0.014*"right" + 0.014*"mitral" + 0.014*"leaflet" + 0.012*"size" + 0.011*"wall" 
Score: 0.1024920791387558	 
Topic: 0.011*"procedur" + 0.008*"patient" + 0.007*"note" + 0.007*"numer" + 0.007*"arteri" + 0.006*"leav" + 0.006*"interpret" + 0.006*"sedat" + 0.006*"place" + 0.006*"id

In [179]:
icd9_suicide_notes2.loc[icd9_suicide_notes2['index'] == 106]

,SUBJECT_ID,ICD9_CODE,ADMITTIME,count,CATEGORY,TEXT,text_processed,index
212,523,E9509,2141-12-19 19:59:00,2,ECG,Sinus rhythm. Non-specific ST-T wave changes....,sinus rhythm non specific st t wave changes co...,106


In [180]:
## determine which topic a certain document belongs to
#106 is index from the suicide group - this is a ECG note specifically for subject 523
for index, score in sorted(lda_model[suicide_bow_corpus[106]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.9097282290458679	 
Topic: 0.018*"plan" + 0.016*"respons" + 0.014*"action" + 0.013*"pain" + 0.013*"assess" + 0.009*"patient" + 0.009*"continu" + 0.009*"monitor" + 0.007*"give" + 0.007*"remain" 
Score: 0.010035047307610512	 
Topic: 0.028*"assess" + 0.018*"lung" + 0.017*"continu" + 0.016*"plan" + 0.014*"sound" + 0.012*"intub" + 0.011*"respons" + 0.011*"ventil" + 0.011*"breath" + 0.011*"secret" 
Score: 0.010032584890723228	 
Topic: 0.014*"patient" + 0.012*"tablet" + 0.009*"daili" + 0.009*"blood" + 0.008*"date" + 0.007*"discharg" + 0.007*"histori" + 0.006*"pain" + 0.006*"follow" + 0.006*"time" 
Score: 0.0100299883633852	 
Topic: 0.011*"procedur" + 0.008*"patient" + 0.007*"note" + 0.007*"numer" + 0.007*"arteri" + 0.006*"leav" + 0.006*"interpret" + 0.006*"sedat" + 0.006*"place" + 0.006*"identifi" 
Score: 0.010029583238065243	 
Topic: 0.010*"right" + 0.009*"patient" + 0.009*"statu" + 0.009*"liver" + 0.008*"continu" + 0.007*"assess" + 0.007*"monitor" + 0.007*"hepat" + 0.006*"histori" +

In [181]:
#this note matches to all ten topics regardless that its a small note
icd9_suicide_notes2['TEXT'].iloc[106]

'Sinus rhythm.  Non-specific ST-T wave changes.  Compared to the previous\ntracing of [**2141-6-28**] no change.\n\n'

In [172]:
## determine which topic a certain document belongs to
#45 is a sample index from the suicide group
for index, score in sorted(lda_model[suicide_bow_corpus[45]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.6095665693283081	 
Topic: 0.018*"plan" + 0.016*"respons" + 0.014*"action" + 0.013*"pain" + 0.013*"assess" + 0.009*"patient" + 0.009*"continu" + 0.009*"monitor" + 0.007*"give" + 0.007*"remain" 
Score: 0.15515859425067902	 
Topic: 0.014*"patient" + 0.012*"tablet" + 0.009*"daili" + 0.009*"blood" + 0.008*"date" + 0.007*"discharg" + 0.007*"histori" + 0.006*"pain" + 0.006*"follow" + 0.006*"time" 
Score: 0.08269412070512772	 
Topic: 0.026*"chest" + 0.023*"reason" + 0.021*"right" + 0.019*"clip" + 0.015*"leav" + 0.013*"tube" + 0.012*"year" + 0.012*"effus" + 0.012*"examin" + 0.011*"report" 
Score: 0.06420174986124039	 
Topic: 0.034*"normal" + 0.029*"valv" + 0.024*"leav" + 0.020*"aortic" + 0.019*"ventricular" + 0.014*"right" + 0.014*"mitral" + 0.014*"leaflet" + 0.012*"size" + 0.011*"wall" 
Score: 0.05120616778731346	 
Topic: 0.028*"assess" + 0.018*"lung" + 0.017*"continu" + 0.016*"plan" + 0.014*"sound" + 0.012*"intub" + 0.011*"respons" + 0.011*"ventil" + 0.011*"breath" + 0.011*"secret" 


In [192]:
icd9_suicide_notes2['TEXT'].iloc[45]

'[**2163-4-3**] 4:29 AM\n CT HEAD W/O CONTRAST                                            Clip # [**Clip Number (Radiology) 38427**]\n Reason: head bleed, fx\n Admitting Diagnosis: TYLENOL,OVERDOSE;LIVER FAILURE\n ______________________________________________________________________________\n [**Hospital 4**] MEDICAL CONDITION:\n  20 year old man with drug overdose, fell out of bed.  Hit head, now with pain\n  in neck and head.\n REASON FOR THIS EXAMINATION:\n  head bleed, fx\n No contraindications for IV contrast\n ______________________________________________________________________________\n WET READ: FKh SUN [**2163-4-3**] 4:47 AM\n  negative.\n ______________________________________________________________________________\n                                 FINAL REPORT\n INDICATION: 20 year old male with drug overdose, fell out of bed and hit his\n head.\n\n TECHNIQUE: Noncontrast head CT.\n\n COMPARISON: Comparison is made to [**2163-3-30**].\n\n FINDINGS: There is no evidence o

In [152]:
#looking to see what subject belongs to this index
icd9_suicide_notes2.iloc[45]

SUBJECT_ID                                                      303
ICD9_CODE                                                     E9500
ADMITTIME                                       2163-03-29 23:04:00
count                                                             2
CATEGORY                                                  Radiology
TEXT              [**2163-4-3**] 4:29 AM\n CT HEAD W/O CONTRAST ...
text_processed          am ct head w o contrast clip clip number...
index                                                            45
Name: 90, dtype: object

In [154]:
#all notes for subject 303
icd9_suicide_notes2.loc[icd9_suicide_notes2['SUBJECT_ID'] == 303][0:5]

,SUBJECT_ID,ICD9_CODE,ADMITTIME,count,CATEGORY,TEXT,text_processed,index
0,303,E9500,2160-03-17 18:37:00,2,Discharge summary,Admission Date: [**2160-3-17**] Discharge...,admission date discharge date date of bi...,0
2,303,E9500,2160-03-17 18:37:00,2,Discharge summary,Admission Date: [**2163-3-29**] ...,admission date discharge date date of bi...,1
4,303,E9500,2160-03-17 18:37:00,2,ECG,Sinus rhythm\nNormal ECG for age\n\n,sinus rhythm normal ecg for age,2
6,303,E9500,2160-03-17 18:37:00,2,ECG,Sinus rhythm. Slight non-specific inferior ST-...,sinus rhythm slight non specific inferior st t...,3
8,303,E9500,2160-03-17 18:37:00,2,Radiology,[**2160-3-17**] 5:16 PM\n TRAUMA SERIES (LAT C...,pm trauma series lat c spine ap cxr ap p...,4


In [193]:
## determine which topic a certain document belongs to
#1 is a sample index from the suicide group - this is a discharge note specifically for subject 303
for index, score in sorted(lda_model[suicide_bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {} ".format(score, lda_model.print_topic(index, 10)))

Score: 0.3352828323841095	 
Topic: 0.018*"plan" + 0.016*"respons" + 0.014*"action" + 0.013*"pain" + 0.013*"assess" + 0.009*"patient" + 0.009*"continu" + 0.009*"monitor" + 0.007*"give" + 0.007*"remain" 
Score: 0.19374202191829681	 
Topic: 0.011*"procedur" + 0.008*"patient" + 0.007*"note" + 0.007*"numer" + 0.007*"arteri" + 0.006*"leav" + 0.006*"interpret" + 0.006*"sedat" + 0.006*"place" + 0.006*"identifi" 
Score: 0.16506941616535187	 
Topic: 0.014*"patient" + 0.012*"tablet" + 0.009*"daili" + 0.009*"blood" + 0.008*"date" + 0.007*"discharg" + 0.007*"histori" + 0.006*"pain" + 0.006*"follow" + 0.006*"time" 
Score: 0.15428926050662994	 
Topic: 0.026*"chest" + 0.023*"reason" + 0.021*"right" + 0.019*"clip" + 0.015*"leav" + 0.013*"tube" + 0.012*"year" + 0.012*"effus" + 0.012*"examin" + 0.011*"report" 
Score: 0.14855054020881653	 
Topic: 0.034*"normal" + 0.029*"valv" + 0.024*"leav" + 0.020*"aortic" + 0.019*"ventricular" + 0.014*"right" + 0.014*"mitral" + 0.014*"leaflet" + 0.012*"size" + 0.011*"wa